In [1]:
import torch
import torch.nn as nn
import torchvision.models as models
import torch.nn.functional as F
import numpy as np

## 依照指示取出模型特定層的資訊

In [2]:
# Define model
class TheModelClass(nn.Module):
    def __init__(self):
        super(TheModelClass, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16* 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Initialize model
model = TheModelClass()


### 取出 self.pool層兩次的輸出，包含：
* x = self.pool(F.relu(self.conv1(x)))
* x = self.pool(F.relu(self.conv2(x)))

In [3]:
for name,_ in model.named_children():
    print(name)

model.children

conv1
pool
conv2
fc1
fc2
fc3


<bound method Module.children of TheModelClass(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)>

In [4]:
outputs= []
def layer1_hook(module, input_, output):
    outputs.append(F.relu(output))

model.conv1.register_forward_hook(layer1_hook)
model.conv2.register_forward_hook(layer1_hook)

In [5]:
input_ = torch.randn(1, 3, 32, 32)
output = model(input_)

In [6]:
outputs

[tensor([[[[0.5198, 0.0000, 0.0000,  ..., 0.0000, 0.7297, 0.0000],
           [0.0000, 0.0000, 0.1840,  ..., 0.4455, 0.0000, 0.0000],
           [0.4289, 0.0000, 0.0000,  ..., 0.2774, 0.5025, 0.2885],
           ...,
           [0.0000, 0.0000, 0.6796,  ..., 0.1752, 0.1801, 0.0000],
           [0.0000, 0.0465, 0.8645,  ..., 0.1453, 0.0000, 0.0455],
           [0.1465, 0.3368, 0.0000,  ..., 0.2942, 0.0000, 0.0000]],
 
          [[0.0000, 0.0000, 0.0746,  ..., 0.1057, 0.1644, 0.9220],
           [0.0000, 1.1292, 0.1774,  ..., 0.6501, 0.6280, 0.0000],
           [0.0000, 0.1598, 0.4203,  ..., 0.0000, 0.1140, 0.0000],
           ...,
           [0.4834, 0.2920, 0.0000,  ..., 0.0038, 0.5905, 0.7987],
           [0.0000, 0.2492, 0.2636,  ..., 1.1121, 0.0000, 0.6071],
           [0.0858, 0.1911, 0.0000,  ..., 0.5022, 1.0636, 0.1988]],
 
          [[0.0000, 1.0403, 0.0452,  ..., 0.1293, 0.6223, 0.0000],
           [0.0000, 0.2887, 0.0000,  ..., 0.1918, 0.0000, 0.0000],
           [0.2023, 0.00

In [7]:
print(outputs[0].shape)
print(outputs[1].shape)

torch.Size([1, 6, 28, 28])
torch.Size([1, 16, 10, 10])


## 加入自定義 initialization fuction

#### 對所有Conv2D層使用自定義initialization function
* weight : nn.init.kaiming_normal_
* bias : 全部輸入1

In [8]:
from torch.nn import init

def weights_init(m):
    #classname = m.__class__.__name__
    if isinstance(m, nn.Conv2d):
        #torch.nn.init.xavier_uniform_(m.weight)
        nn.init.kaiming_normal_(m.weight.data,
                                    a=0,
                                    mode='fan_out',
                                    nonlinearity='relu')
        if m.bias is not None:
            torch.nn.init.zeros_(m.bias)
            
model.apply(weights_init)

TheModelClass(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

#### 查看 conv層的bias是否皆為1

In [9]:
for name, parameters in model.named_parameters():
    if ('conv' in name) and ('bias' in name):
        print(name, parameters)
        print('\n')

conv1.bias Parameter containing:
tensor([0., 0., 0., 0., 0., 0.], requires_grad=True)


conv2.bias Parameter containing:
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       requires_grad=True)


